<a href="https://colab.research.google.com/github/parth06/Fake-News-Challenge/blob/master/Neural%20Net/fnc_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/parth06/Fake-News-Challenge.git

In [0]:
cd Fake-News-Challenge/

In [0]:
!git submodule init
!git submodule update

In [0]:
import nltk

In [0]:
nltk.download('punkt')
nltk.download('wordnet')

In [0]:
import pandas as pd
import numpy as np

In [0]:
# fix random seed for reproducibility
seed = 1
np.random.seed(seed)

In [0]:
#the data directory
DATA_DIR = './data'

In [0]:
# Read the text files of fnc data
bodies = pd.read_csv(DATA_DIR + '/body_id.csv')
train_df = pd.read_csv(DATA_DIR + '/train.csv')
validation_df = pd.read_csv(DATA_DIR + '/validation.csv')
test_df = pd.read_csv(DATA_DIR + '/test.csv')

In [0]:
train_df.shape

In [0]:
bodies.head()

In [0]:
train_df.head()

In [0]:
train_df["Stance"].head()#value_counts()

In [0]:
labels = train_df['Stance'].astype('category').cat.categories.tolist()
replace_category = {'Stance' : {k: v for k,v in zip(labels,list(range(1,len(labels)+1)))}}
reverse_labels = {'Stance' : {v: k for k,v in zip(labels,list(range(1,len(labels)+1)))}}

print(replace_category)

In [0]:
train_df.replace(replace_category, inplace=True)
validation_df.replace(replace_category, inplace=True)

In [0]:
train_df.head()

In [0]:
train = train_df.merge(bodies,on="Body ID")
val = validation_df.merge(bodies,on="Body ID")
test = test_df.merge(bodies,on="Body ID")

In [0]:
# Xtest_overlap = gen_or_load_feats(word_overlap_features, test["Headline"], test["articleBody"], "features/overlap.train.npy")
# Xtest_refuting = gen_or_load_feats(refuting_features,test["Headline"], test["articleBody"], "features/refuting.train.npy")
# Xtest_polarity = gen_or_load_feats(polarity_features, test["Headline"], test["articleBody"], "features/polarity.train.npy")
# Xtest_hand = gen_or_load_feats(hand_features, test["Headline"], test["articleBody"], "features/hand.train.npy")

In [0]:
# train.drop(labels="Body ID",axis=1,inplace=True)
# val.drop(labels="Body ID",axis=1,inplace=True)
# test.drop(labels="Body ID",axis=1,inplace=True)

In [0]:
train.head()

,Headline,Body ID,Stance,articleBody
0,Hundreds of Palestinians flee floods in Gaza a...,158,1,Hundreds of Palestinians were evacuated from t...
1,It's 'rubbish' that Robert Plant turned down £...,158,4,Hundreds of Palestinians were evacuated from t...
2,Robert Plant ripped up $800M Led Zeppelin reun...,158,4,Hundreds of Palestinians were evacuated from t...
3,ISIS Militant “Jihadi John” Identified As Youn...,158,4,Hundreds of Palestinians were evacuated from t...
4,Claim: Comcast Got Complaining Customer Fired ...,158,4,Hundreds of Palestinians were evacuated from t...


In [0]:
from keras.preprocessing.text import text_to_word_sequence, Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [0]:
# Pre-processing involves removal of puctuations and converting text to lower case
train["Headline"] = [text_to_word_sequence(head) for head in train['Headline']]
train["articleBody"] = [text_to_word_sequence(body) for body in train['articleBody']]
val["Headline"] = [text_to_word_sequence(head) for head in val['Headline']]
val["articleBody"] = [text_to_word_sequence(body) for body in val['articleBody']]
test["Headline"] = [text_to_word_sequence(head) for head in test['Headline']]
test["articleBody"] = [text_to_word_sequence(body) for body in test['articleBody']]

In [0]:
train.Headline = train.Headline.apply(" ".join)
train.articleBody = train.articleBody.apply(" ".join)
val.Headline = val.Headline.apply(" ".join)
val.articleBody = val.articleBody.apply(" ".join)
test.Headline = test.Headline.apply(" ".join)
test.articleBody = test.articleBody.apply(" ".join)


In [0]:
#Unrelated:1 Related:0
def f(row):
    if row['Stance']==4:
      return 1
    else:
      return 0

train['Classify'] = train.apply(f, axis=1)
val['Classify'] = val.apply(f, axis=1)

In [0]:
train["Classify"].value_counts()

1    29647
0    10703
Name: Classify, dtype: int64

In [0]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

In [0]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2"

In [0]:
train["inner"]=-1

In [0]:
bodies["emb"] = -1
bodies.head()

,Body ID,articleBody,emb
0,0,A small meteorite crashed into a wooded area i...,-1
1,1,Al-Sisi has denied Israeli reports stating tha...,-1
2,2,A bereaved Afghan mother took revenge on the T...,-1
3,3,CNBC is reporting Tesla has chosen Nevada as t...,-1
4,4,Last week we hinted at what was to come as Ebo...,-1


In [0]:
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)

In [0]:
embed.get_attached_message

<bound method Module.get_attached_message of <tensorflow_hub.module.Module object at 0x7fd84fb83da0>>

In [0]:
# Compute a representation for each message, showing various lengths supported.
l = len(train)
#for i in range(0,l):
messages = [train["Headline"][i] for i in range(0,l)]
# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)
emb = list()
with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  message_embeddings = session.run(embed(messages))

  for j, message_embedding in enumerate(np.array(message_embeddings).tolist()):
    emb.append(message_embedding)



In [0]:
len(emb)

40350

In [0]:
# Compute a representation for each message, showing various lengths supported.
l = len(bodies)
#for i in range(0,l):
messages = [bodies["articleBody"][i][:300] for i in range(0,l)]
# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)
emb2 = list()
with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  message_embeddings = session.run(embed(messages))

  for j, message_embedding in enumerate(np.array(message_embeddings).tolist()):
    emb2.append(message_embedding)


In [0]:
train["embh"] = emb
bodies["embb"] = emb2

In [0]:
df = pd.merge(train,bodies[['Body ID','embb']],on='Body ID', how='left')
df.head()

,Headline,Body ID,Stance,articleBody,Classify,embh,embb
0,hundreds of palestinians flee floods in gaza a...,158,1,hundreds of palestinians were evacuated from t...,0,"[-0.05026805400848389, -0.005613733548671007, ...","[-0.035615868866443634, -0.04305685684084892, ..."
1,it's 'rubbish' that robert plant turned down £...,158,4,hundreds of palestinians were evacuated from t...,1,"[0.06691830605268478, 0.05369390547275543, 0.0...","[-0.035615868866443634, -0.04305685684084892, ..."
2,robert plant ripped up 800m led zeppelin reuni...,158,4,hundreds of palestinians were evacuated from t...,1,"[0.044021785259246826, 0.010502968914806843, -...","[-0.035615868866443634, -0.04305685684084892, ..."
3,isis militant “jihadi john” identified as youn...,158,4,hundreds of palestinians were evacuated from t...,1,"[0.03243670612573624, -0.06309863179922104, -0...","[-0.035615868866443634, -0.04305685684084892, ..."
4,claim comcast got complaining customer fired f...,158,4,hundreds of palestinians were evacuated from t...,1,"[0.020369600504636765, 0.05546443909406662, -0...","[-0.035615868866443634, -0.04305685684084892, ..."


In [0]:
embb = list(df.loc[:,"embb"])

In [0]:
import gensim
import numpy as np
import pandas as pd
import os
import re
import csv
import codecs
import sys
import pickle

from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split

from keras import callbacks
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout, BatchNormalization, Activation, Bidirectional
from keras.layers.merge import concatenate
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.text import text_to_word_sequence, Tokenizer
from keras.preprocessing.sequence import pad_sequences

import matplotlib as mpl
%matplotlib inline
from matplotlib import pyplot as plt
from keras.utils import plot_model 
from IPython.display import Image
import pydot
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
import tensorflow as tf


from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

from string import punctuation

from keras import backend as K
from keras.engine.topology import Layer
#from keras import initializations
from keras import initializers, regularizers, constraints

tf_gpu_options = tf.GPUOptions(allow_growth = True) # per_process_gpu_memory_fraction=0.12,
tf_session = tf.Session(config=tf.ConfigProto(gpu_options=tf_gpu_options))
tf.keras.backend.set_session(tf_session)

#biodirectional embedding


Using TensorFlow backend.


In [0]:
from keras.utils import plot_model
plot_model(merged_model, show_shapes=True)

In [0]:
model1_in = Input(shape=(512,))
model1_out = (model1_in)
model1 = Model(model1_in, model1_out)

model2_in = Input(shape=(512,))
model2_out = (model2_in)
model2 = Model(model2_in, model2_out)


concatenated = concatenate([model1_out, model2_out])
layer_1 = Dense(256, activation='relu', name='layer_1') (concatenated)
dropout_1 = Dropout(rate=0.5, name='dropout_1')(layer_1)
layer_2 =Dense(64,  activation='relu', name='layer_2') (dropout_1)
dropout_2 = Dropout(rate=0.5, name='dropout_2')(layer_2)
out1 =  Dense(2, activation='softmax', name='output_layer1')(dropout_2)
out2 = Dense(4, activation='softmax', name='output_layer2')(dropout_2)

losses = {
	"output_layer1": "categorical_crossentropy",
	"output_layer2": "categorical_crossentropy",
}
lossWeights = {"output_layer1": 0.25, "output_layer2": 0.75}
               
merged_model = Model([model1_in, model2_in], [out1,out2])
merged_model.compile(loss=losses, optimizer='adam', loss_weights=lossWeights,
metrics=['accuracy'])

In [0]:
merged_model.fit([emb, embb], y=[dummy_y_train,dummy_y1_train], batch_size=128, epochs=20, 
               validation_split=0.1,verbose=1, shuffle=True)

Train on 36315 samples, validate on 4035 samples
Epoch 1/20
36315/36315 [==============================] - 6s 169us/step - loss: 0.5377 - output_layer1_loss: 0.4156 - output_layer2_loss: 0.5783 - output_layer1_acc: 0.8021 - output_layer2_acc: 0.7927 - val_loss: 0.6331 - val_output_layer1_loss: 0.3380 - val_output_layer2_loss: 0.7315 - val_output_layer1_acc: 0.9036 - val_output_layer2_acc: 0.7093
Epoch 2/20
36315/36315 [==============================] - 3s 73us/step - loss: 0.2688 - output_layer1_loss: 0.1565 - output_layer2_loss: 0.3062 - output_layer1_acc: 0.9409 - output_layer2_acc: 0.8886 - val_loss: 0.4686 - val_output_layer1_loss: 0.1630 - val_output_layer2_loss: 0.5705 - val_output_layer1_acc: 0.9477 - val_output_layer2_acc: 0.7735
Epoch 3/20
36315/36315 [==============================] - 3s 75us/step - loss: 0.2093 - output_layer1_loss: 0.1032 - output_layer2_loss: 0.2446 - output_layer1_acc: 0.9615 - output_layer2_acc: 0.9117 - val_loss: 0.4738 - val_output_layer1_loss: 0.1705 

In [0]:
y_train = df['Stance']
#y_val = combine_df_validation['Stance']
#Encode class values as integers
encoder_train = LabelEncoder()
encoder_train.fit(y_train)
encoded_train = encoder_train.transform(y_train)
#encoded_val = encoder_train.transform(y_val)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y1_train = np_utils.to_categorical(encoded_train)
#dummy_y_val = np_utils.to_categorical(encoded_val)

In [0]:
dummy_y1_train

array([[1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       ...,
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.]], dtype=float32)

In [0]:
l = len(df)
ans = list()
for i in range(0,l):
  ans.append(np.inner(df.embh[i],df.embb[i]))

In [0]:
df.inner = ans
df.head()

,Headline,Body ID,Stance,articleBody,Classify,inner,embh,embb
0,hundreds of palestinians flee floods in gaza a...,158,1,hundreds of palestinians were evacuated from t...,0,0.815439,"[-0.05026805400848389, -0.005613733548671007, ...","[-0.0466877780854702, -0.010597342625260353, -..."
1,it's 'rubbish' that robert plant turned down £...,158,4,hundreds of palestinians were evacuated from t...,1,0.242716,"[0.06691830605268478, 0.05369390547275543, 0.0...","[-0.0466877780854702, -0.010597342625260353, -..."
2,robert plant ripped up 800m led zeppelin reuni...,158,4,hundreds of palestinians were evacuated from t...,1,0.114176,"[0.044021785259246826, 0.010502968914806843, -...","[-0.0466877780854702, -0.010597342625260353, -..."
3,isis militant “jihadi john” identified as youn...,158,4,hundreds of palestinians were evacuated from t...,1,0.315851,"[0.03243670612573624, -0.06309863179922104, -0...","[-0.0466877780854702, -0.010597342625260353, -..."
4,claim comcast got complaining customer fired f...,158,4,hundreds of palestinians were evacuated from t...,1,0.188835,"[0.020369600504636765, 0.05546443909406662, -0...","[-0.0466877780854702, -0.010597342625260353, -..."


In [0]:
df[df.inner<0.45]["Classify"].value_counts()

1    25824
0      429
Name: Classify, dtype: int64

In [0]:
df[df.inner>0.45]["Stance"].value_counts()

3    6886
4    3823
1    2758
2     630
Name: Stance, dtype: int64

In [0]:
# indexNames = df[ df.inner<0.45 ].index
 
# # Delete these row indexes from dataFrame
# df.drop(indexNames , inplace=True)

In [0]:
#from sklearn.metrics.pairwise import cosine_similarity

In [0]:
# Compute a representation for each message, showing various lengths supported.
l = len(test)
#for i in range(0,l):
messages = [test["Headline"][i] for i in range(0,l)]
# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)
emb3 = list()
with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  message_embeddings = session.run(embed(messages))

  for j, message_embedding in enumerate(np.array(message_embeddings).tolist()):
    emb3.append(message_embedding)



In [0]:
test["embh"] = emb3

In [0]:
test = pd.merge(test,bodies[['Body ID','embb']],on='Body ID', how='left')

In [0]:
embbt = list(test.loc[:,"embb"])

In [0]:
a,b = merged_model.predict([emb3,embbt])

In [0]:
a = np.argmax(a,axis=1)
b = np.argmax(b,axis=1)
pred=list()
for i in range(0,len(a)):
  if a[i]==1: #unrelated
    pred.append(3)
  elif a[i]==0 and b[i]==4:
    pred.append(4)
  else:
    pred.append(b[i])
    

In [0]:
ans = pd.DataFrame(pred)
ans.replace(3,'unrelated',True)
ans.replace(0,'agree',True)
ans.replace(1,'disagree',True)
ans.replace(2,'discuss',True)
ans.head()

,0
0,discuss
1,unrelated
2,unrelated
3,unrelated
4,discuss


In [0]:
import sys
import numpy as np

from sklearn.ensemble import GradientBoostingClassifier
from feature_engineering import refuting_features, polarity_features, hand_features, gen_or_load_feats
from feature_engineering import word_overlap_features
from utils.dataset import DataSet
from utils.generate_test_splits import kfold_split, get_stances_for_folds
from utils.score import report_score, LABELS, score_submission

from utils.system import parse_params, check_version
def generate_features(stances,dataset,name):
    h, b, y = [],[],[]

    for stance in stances:
        y.append(LABELS.index(stance['Stance']))
        h.append(stance['Headline'])
        b.append(dataset.articles[stance['Body ID']])

    return y
  

competition_dataset = DataSet("competition_test")
y_competition = generate_features(competition_dataset.stances, competition_dataset, "competition")

Xs = dict()
ys = dict()

# Load/Precompute all features now
#Run on competition dataset
actual = [LABELS[int(a)] for a in y_competition]

print("Scores on the test set")
report_score(actual,ans[0])


Reading dataset
Total stances: 25413
Total bodies: 904
Scores on the test set
-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    216    |     0     |    373    |   1314    |
-------------------------------------------------------------
| disagree  |    107    |     0     |    150    |    440    |
-------------------------------------------------------------
|  discuss  |    432    |     0     |    775    |   3257    |
-------------------------------------------------------------
| unrelated |   1766    |     0     |   3078    |   13505   |
-------------------------------------------------------------
Score: 4632.75 out of 11651.25	(39.76182813002897%)


39.76182813002897

In [0]:
l = len(test)
anst = list()
for i in range(0,l):
  anst.append(np.inner(test.embh[i],test.embb[i]))
test["inner"] = anst

In [0]:
test["Stance"]=-1
for i in range(0,len(test)):
  if test.inner[i] <0.45:
    test.loc[i,"Stance"]=4

In [0]:
test.head()

,Headline,Body ID,articleBody,embh,embb,inner,Stance
0,ferguson riots pregnant woman loses eye after ...,2008,a respected senior french police officer inves...,"[-0.06295444816350937, -0.07035506516695023, -...","[0.0317782461643219, -0.050620388239622116, -0...",0.523870,-1
1,apple stores to install safes to secure gold a...,2008,a respected senior french police officer inves...,"[0.03951696306467056, -0.049585841596126556, -...","[0.0317782461643219, -0.050620388239622116, -0...",0.157018,4
2,pregnant woman loses eye after police shoot be...,2008,a respected senior french police officer inves...,"[-0.07143513113260269, -0.0795423835515976, -0...","[0.0317782461643219, -0.050620388239622116, -0...",0.420942,4
3,we just found out the ferguson protester who c...,2008,a respected senior french police officer inves...,"[-0.04828790947794914, 0.009491262026131153, -...","[0.0317782461643219, -0.050620388239622116, -0...",0.466931,-1
4,police chief in charge of paris attacks commit...,2008,a respected senior french police officer inves...,"[0.02026490867137909, -0.05242277309298515, -0...","[0.0317782461643219, -0.050620388239622116, -0...",0.730503,-1


In [0]:
predicted = [int(a) for a in clf.predict(X)]

In [0]:
j=0
for i in range(0,len(test)):
  if test.Stance[i]==-1:
    if test.inner > 0.85 and predicted[j]==4:
      test.loc[i,"Stance"]=3
    else:
      test.loc[i,"Stance"]=predicted[j]
    j=j+1

In [0]:
test.Stance.value_counts()

unrelated    19475
discuss       5519
agree          395
disagree        24
Name: Stance, dtype: int64

In [0]:
# labels = ['Stance'].astype('category').cat.categories.tolist()
# replace_category = {'Stance' : {k: v for k,v in zip(labels,list(range(1,len(labels)+1)))}}
# reverse_labels = {'Stance' : {v: k for k,v in zip(labels,list(range(1,len(labels)+1)))}}

# print(replace_category)

test.replace(reverse_labels, inplace=True)

In [0]:
import sys
import numpy as np

from sklearn.ensemble import GradientBoostingClassifier
from feature_engineering import refuting_features, polarity_features, hand_features, gen_or_load_feats
from feature_engineering import word_overlap_features
from utils.dataset import DataSet
from utils.generate_test_splits import kfold_split, get_stances_for_folds
from utils.score import report_score, LABELS, score_submission

from utils.system import parse_params, check_version
def generate_features(stances,dataset,name):
    h, b, y = [],[],[]

    for stance in stances:
        y.append(LABELS.index(stance['Stance']))
        h.append(stance['Headline'])
        b.append(dataset.articles[stance['Body ID']])

    return y
  

competition_dataset = DataSet("competition_test")
y_competition = generate_features(competition_dataset.stances, competition_dataset, "competition")

Xs = dict()
ys = dict()


Reading dataset
Total stances: 25413
Total bodies: 904


In [0]:
#Run on competition dataset
actual = [LABELS[int(a)] for a in y_competition]

print("Scores on the test set")
report_score(actual,pred)


Scores on the test set
-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    33     |     1     |    454    |   1415    |
-------------------------------------------------------------
| disagree  |    17     |     2     |    201    |    477    |
-------------------------------------------------------------
|  discuss  |    64     |     6     |    995    |   3399    |
-------------------------------------------------------------
| unrelated |    281    |    15     |   3869    |   14184   |
-------------------------------------------------------------
Score: 4761.75 out of 11651.25	(40.86900547151593%)


40.86900547151593

,Headline,Body ID,Stance,articleBody,Classify,inner,embh
0,hundreds of palestinians flee floods in gaza a...,158,1,hundreds of palestinians were evacuated from t...,0,-1,"[-0.05026805400848389, -0.005613733548671007, ..."
1,it's 'rubbish' that robert plant turned down £...,158,4,hundreds of palestinians were evacuated from t...,1,-1,"[0.06691830605268478, 0.05369390547275543, 0.0..."
2,robert plant ripped up 800m led zeppelin reuni...,158,4,hundreds of palestinians were evacuated from t...,1,-1,"[0.044021785259246826, 0.010502968914806843, -..."
3,isis militant “jihadi john” identified as youn...,158,4,hundreds of palestinians were evacuated from t...,1,-1,"[0.03243670612573624, -0.06309863179922104, -0..."
4,claim comcast got complaining customer fired f...,158,4,hundreds of palestinians were evacuated from t...,1,-1,"[0.020369600504636765, 0.05546443909406662, -0..."


In [0]:
from keras.models import Sequential
from keras.layers import LSTM
from numpy import array
# define model where LSTM is also output layer
model1 = Sequential()
model1.add(LSTM(1, input_shape=(1,512)))
model1.compile(optimizer='adam', loss='mse')
# input time steps
# make and show prediction
print(model1.predict(train[["embh","Stance"]]))

In [0]:
nltk.download('stopwords')
from nltk.corpus import stopwords 
stop_words = stopwords.words('english')

In [0]:
train["Headline"] = train["Headline"].apply(lambda x: [item for item in x if item not in stop_words])
train["articleBody"] =train['articleBody'].apply(lambda x: [item for item in x if item not in stop_words])
val["Headline"] = val['Headline'].apply(lambda x: [item for item in x if item not in stop_words])
val["articleBody"] = val['articleBody'].apply(lambda x: [item for item in x if item not in stop_words])
test["Headline"] = test['Headline'].apply(lambda x: [item for item in x if item not in stop_words])
test["articleBody"] = test['articleBody'].apply(lambda x: [item for item in x if item not in stop_words])

In [0]:
train.head()

In [0]:
print('90th Percentile Sentence of headline:', np.percentile([len(seq) for seq in train["Headline"]], 90))
print('90th Percentile Sentence of body:', np.percentile([len(seq) for seq in train["articleBody"]], 50))